In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # force TensorFlow à utiliser le CPU

import tensorflow as tf
print("GPU disponible :", tf.config.list_physical_devices('GPU'))  # devrait être vide pour CPU

GPU disponible : []


In [3]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.read_csv('canadianHousePrice.csv')
df1.head()

,rentfaster_id,city,province,address,latitude,longitude,lease_term,type,price,beds,baths,sq_feet,link,furnishing,availability_date,smoking,cats,dogs
0,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
1,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2695.0,3 Beds,2.5,1496,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
2,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2295.0,2 Beds,2.5,1180,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True
3,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2095.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,November 18,Non-Smoking,True,True
4,468622,Airdrie,Alberta,69 Gateway Dr NE,51.305962,-114.012515,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,/ab/airdrie/rentals/townhouse/2-bedrooms/pet-f...,Unfurnished,Immediate,Non-Smoking,True,True


In [5]:
df2 = df1.drop(['rentfaster_id','address','latitude', 'longitude', 'link', 'availability_date'], axis= 'columns')
df2.isnull().sum()

city             0
province         0
lease_term      46
type             0
price            0
beds           132
baths          134
sq_feet       4112
furnishing       0
smoking       2702
cats           173
dogs           173
dtype: int64

In [6]:
df2.shape

(25771, 12)

In [7]:
df2['beds'] = df2['beds'].fillna(0)
df2['baths'] = df2['baths'].fillna(0)
df2['cats'] = df2['cats'].fillna(False)
df2['dogs'] = df2['dogs'].fillna(False)
df2.isnull().sum()

/tmp/ipykernel_76375/4159675197.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2['cats'] = df2['cats'].fillna(False)
/tmp/ipykernel_76375/4159675197.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2['dogs'] = df2['dogs'].fillna(False)


city             0
province         0
lease_term      46
type             0
price            0
beds             0
baths            0
sq_feet       4112
furnishing       0
smoking       2702
cats             0
dogs             0
dtype: int64

In [8]:
df3 = df2.dropna()
df3.isnull().sum()

city          0
province      0
lease_term    0
type          0
price         0
beds          0
baths         0
sq_feet       0
furnishing    0
smoking       0
cats          0
dogs          0
dtype: int64

In [9]:
df3.shape
df3.head(1)

,city,province,lease_term,type,price,beds,baths,sq_feet,furnishing,smoking,cats,dogs
0,Airdrie,Alberta,Long Term,Townhouse,2495.0,2 Beds,2.5,1403,Unfurnished,Non-Smoking,True,True


In [10]:
df3['baths'].unique()

array(['2.5', '1', '2', '1.5', '3.5', '4', '3', '5', 'none', '4.5', '7.5',
       0, '5.5', '6', '6.5', '0', '7'], dtype=object)

In [11]:
def convertBedsToNumber(x):
    if pd.isna(x):
        return 0
    if isinstance(x, (int, float)):
        return int(x)
    
    first = str(x).split(' ')[0].lower()
    if first in ['none', 'studio']:
        return 0
    try:
        return int(first)
    except ValueError:
        return 0 

In [12]:
df3.loc[:,'nbr_beds'] = df3['beds'].apply(convertBedsToNumber)
df3.drop(['beds'], axis = 'columns')

/tmp/ipykernel_76375/3838658798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:,'nbr_beds'] = df3['beds'].apply(convertBedsToNumber)


,city,province,lease_term,type,price,baths,sq_feet,furnishing,smoking,cats,dogs,nbr_beds
0,Airdrie,Alberta,Long Term,Townhouse,2495.0,2.5,1403,Unfurnished,Non-Smoking,True,True,2
1,Airdrie,Alberta,Long Term,Townhouse,2695.0,2.5,1496,Unfurnished,Non-Smoking,True,True,3
2,Airdrie,Alberta,Long Term,Townhouse,2295.0,2.5,1180,Unfurnished,Non-Smoking,True,True,2
3,Airdrie,Alberta,Long Term,Townhouse,2095.0,2.5,1403,Unfurnished,Non-Smoking,True,True,2
4,Airdrie,Alberta,Long Term,Townhouse,2495.0,2.5,1403,Unfurnished,Non-Smoking,True,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...
25766,Yorkton,Saskatchewan,Long Term,Apartment,920.0,1,400,Unfurnished,Non-Smoking,True,True,0
25767,Yorkton,Saskatchewan,Long Term,Apartment,1450.0,1,810,Unfurnished,Non-Smoking,True,True,3
25768,Yorkton,Saskatchewan,Long Term,Apartment,1125.0,1,520,Unfurnished,Non-Smoking,True,True,1
25769,Yorkton,Saskatchewan,Long Term,Apartment,920.0,1,400,Unfurnished,Non-Smoking,True,True,0


In [13]:
def boolToNumber(x):
    if x == True:
        return 1
    if x == False:
        return 0

In [14]:
df3['cats'] = df3['cats'].apply(boolToNumber)
df3['dogs'] = df3['dogs'].apply(boolToNumber)

/tmp/ipykernel_76375/139235915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['cats'] = df3['cats'].apply(boolToNumber)
/tmp/ipykernel_76375/139235915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['dogs'] = df3['dogs'].apply(boolToNumber)


In [15]:
count_location = df3.groupby('city')['city'].agg('count').sort_values(ascending =  False)
count_location

city
Calgary       7828
Edmonton      2272
Toronto       1575
Montréal      1044
Ottawa         840
              ... 
Tsawwassen       1
Wardner          1
Vanier           1
Westerose        1
Whitecourt       1
Name: city, Length: 233, dtype: int64

In [16]:
location_less_than_10 = count_location[count_location<=100]
location_less_than_10

city
Laval           86
Oshawa          83
Windsor         80
Medicine Hat    75
Kelowna         74
                ..
Tsawwassen       1
Wardner          1
Vanier           1
Westerose        1
Whitecourt       1
Name: city, Length: 213, dtype: int64

In [17]:
df3.city = df3.city.apply(lambda x: 'others' if x in location_less_than_10 else x)

/tmp/ipykernel_76375/2586879933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.city = df3.city.apply(lambda x: 'others' if x in location_less_than_10 else x)


In [18]:
def convertToFlaot(x):
    return float(x)

In [19]:
df3 = df3[pd.to_numeric(df3['sq_feet'], errors='coerce').notna()]

In [20]:
df3['sq_feet'] =df3.sq_feet.apply(convertToFlaot)

In [21]:
df4 = df3[~(df3.sq_feet == 0)]
df4 = df4[~(df4.price == 0)]
df4['price_per_sqare_foot'] = df4.price*100000/df4.sq_feet

In [22]:
def remove_outlier(df):
    df_out = pd.DataFrame()
    for key, sub_df in df.groupby('city'):
        m = np.mean(sub_df.price_per_sqare_foot)
        st = np.std(sub_df.price_per_sqare_foot)
        reduced_df = sub_df[(sub_df.price_per_sqare_foot> (m-st)) & (sub_df.price_per_sqare_foot< (m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index = True)
    return df_out
df5 = remove_outlier(df4)

In [23]:
df5.shape

(16862, 14)

In [24]:
df5 =df5.drop(['lease_term', 'furnishing'], axis = 'columns')

In [25]:
df5.head()

,city,province,type,price,beds,baths,sq_feet,smoking,cats,dogs,nbr_beds,price_per_sqare_foot
0,Airdrie,Alberta,Townhouse,2495.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,2,177833.214540
1,Airdrie,Alberta,Townhouse,2695.0,3 Beds,2.5,1496.0,Non-Smoking,1,1,3,180147.058824
2,Airdrie,Alberta,Townhouse,2295.0,2 Beds,2.5,1180.0,Non-Smoking,1,1,2,194491.525424
3,Airdrie,Alberta,Townhouse,2095.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,2,149322.879544
4,Airdrie,Alberta,Townhouse,2495.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,2,177833.214540


In [26]:
city_dummies = pd.get_dummies(df5.city)
province_dummies = pd.get_dummies(df5.province)
type_dummies = pd.get_dummies(df5.type)
smoking_dummies = pd.get_dummies(df5.smoking)
df6 = pd.concat([df5, city_dummies,province_dummies,type_dummies,smoking_dummies], axis = 'columns')
df6.head()

,city,province,type,price,beds,baths,sq_feet,smoking,cats,dogs,...,Mobile,Office Space,Parking Spot,Room For Rent,Townhouse,Vacation Home,Negotiable,Non-Smoking,Smoke Free Building,Smoking Allowed
0,Airdrie,Alberta,Townhouse,2495.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,...,False,False,False,False,True,False,False,True,False,False
1,Airdrie,Alberta,Townhouse,2695.0,3 Beds,2.5,1496.0,Non-Smoking,1,1,...,False,False,False,False,True,False,False,True,False,False
2,Airdrie,Alberta,Townhouse,2295.0,2 Beds,2.5,1180.0,Non-Smoking,1,1,...,False,False,False,False,True,False,False,True,False,False
3,Airdrie,Alberta,Townhouse,2095.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,...,False,False,False,False,True,False,False,True,False,False
4,Airdrie,Alberta,Townhouse,2495.0,2 Beds,2.5,1403.0,Non-Smoking,1,1,...,False,False,False,False,True,False,False,True,False,False


In [27]:
df7 = df6.drop(['city', 'province','type','smoking'], axis = 'columns')
df7 = df7[pd.to_numeric(df7['baths'], errors='coerce').notna()]


In [28]:
X = df7.drop(['beds','price'], axis = 'columns')
y = df7.price
X.head()


,baths,sq_feet,cats,dogs,nbr_beds,price_per_sqare_foot,Airdrie,Brampton,Calgary,Edmonton,...,Mobile,Office Space,Parking Spot,Room For Rent,Townhouse,Vacation Home,Negotiable,Non-Smoking,Smoke Free Building,Smoking Allowed
0,2.5,1403.0,1,1,2,177833.214540,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False
1,2.5,1496.0,1,1,3,180147.058824,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False
2,2.5,1180.0,1,1,2,194491.525424,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False
3,2.5,1403.0,1,1,2,149322.879544,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False
4,2.5,1403.0,1,1,2,177833.214540,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False


In [29]:

bool_cols = X.select_dtypes(include='bool').columns
X[bool_cols] = X[bool_cols].astype(int)
bool_cols = X.select_dtypes(include='bool').columns
X[bool_cols] = X[bool_cols].astype(int)
X = X.astype('float32')
y = y.astype('float32')
#X.to_csv('X.csv', index=False)  # index=False pour ne pas inclure l'index

# Exporter y si nécessaire
#y.to_csv('y.csv', index=False)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 10)

In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [32]:
model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)   # output price
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64
)

Epoch 1/100


/home/brice/Documents/deepLearning/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


169/169 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6755711.0000 - mae: 1148.4680 - val_loss: 667688.9375 - val_mae: 598.4446
Epoch 2/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 829750.4375 - mae: 647.1064 - val_loss: 497553.1875 - val_mae: 507.9712
Epoch 3/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 511057.9375 - mae: 506.4391 - val_loss: 215223.9531 - val_mae: 332.4196
Epoch 4/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 436397.1875 - mae: 461.8311 - val_loss: 194628.4688 - val_mae: 299.5013
Epoch 5/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 355999.5625 - mae: 416.2563 - val_loss: 198517.7344 - val_mae: 297.9429
Epoch 6/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 311487.2188 - mae: 372.3397 - val_loss: 199107.9219 - val_mae: 336.0250
Epoch 7/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 389909.9062 - mae: 425.5061 - val_loss: 524370.1250 - val_mae: 549.6226
Epoch 8/100
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 348071.4688 - mae:

In [33]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"MSE (erreur quadratique moyenne) : {test_loss:.2f}")
print(f"MAE (erreur absolue moyenne) : {test_mae:.2f}")


106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - loss: 207513.1250 - mae: 262.0153
MSE (erreur quadratique moyenne) : 207513.12
MAE (erreur absolue moyenne) : 262.02


In [34]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² : {r2:.3f}")


106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
R² : 0.725


In [36]:
# Sauvegarder le modèle complet
model.save("price_prediction_model.keras")